Deep Learning
=============

Assignment 6 - PROBLEM 1
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
import time
def how_long(f, *args):
    #medir el tiempo que tarda f
    t1 = time.time()
    res = f(*args)
    t2 = time.time()
    print ("tiempo utilizado = ",t2-t1)
    #return res, t2-t1
    return res

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

Create a small validation set.

In [5]:
valid_size = 1000 #Los mil primeros chars son de validación
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [7]:
batch_size=64 #trocitos de frases en un lote
num_unrollings=10 #caracteres-1 en cada trocito (hay uno más de solapamiento)

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size #reparte el texto en N caminos de predicion (N=batch_size)
                                            #y cada una la va encadenando por separado
        
    self._cursor = [ offset * segment for offset in range(batch_size)] #lleva un array de N cursores (N=batch_size)
                                                                        #para cada camino de prediccion
    self._last_batch = self._next_batch() #fija el último lote al primero de todos
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0 #1-hot-encoded array
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size #incrementa el puntero en modo circular por el texto
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch] #cada trocito de frase del lote se solapa en el último caracter 
                                    #del trocito de frase de la misma posicion en el lote anterior
    for step in range(self._num_unrollings): #va añadiendo chars a cada trocito de texto del lote hasta llegar a num_unrollings+1
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)] #Toma el mayor valor (o valores si hay varios iguales)
                                                            #del vector de probabilidades y devuelve la lista de caracteres

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0] #reserva una cadena vacía para N caminos de predicción (N=batch_size)
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))] #va pegando la cadena predicha en cada batch, o bien el hot-encoded literal
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print("train")
t1=train_batches.next()
print(batches2string(t1))
print(len(t1))
print(t1[0].shape)
print(batches2string(train_batches.next()))
print("\nvalid")
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))

train
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
11
(64, 27)
['ists advoca', 'ary 

Minimize Average Negative Log-Probability:
$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}$$

Measure perplexity:

$$e^{-\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}} = e^{\text{loss}}$$

In [8]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized probabilities."""
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

[documento] (http://arxiv.org/pdf/1402.1128v1.pdf)

NOTAS: 
<img src="LSTM_notas.jpg">

In [9]:
##################
# DECLARACION LSTM1
##################

num_nodes = 64 #Tantas células en la memoria como líneas de entrenamiento va a seguir en un lote, que casualidad
starter_learning_rate = 10.0
learning_decay_steps = 5000
learning_decay_rate = 0.1
clip_limit = 1.25

graphLSTM1 = tf.Graph()
with graphLSTM1.as_default():
  
  # Parameters:
  
  # Gates: 0=memory cell, 1=input, 2=forget, 3=output
  num_gates = 4  

  #Gates parameters for input:
  #ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  #ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  gx = tf.Variable(tf.truncated_normal([num_gates, vocabulary_size, num_nodes], -0.1, 0.1))

  #Gates parameters for output:
  #im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  #cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  #fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  #om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  gm = tf.Variable(tf.truncated_normal([num_gates, num_nodes, num_nodes], -0.1, 0.1))
    
  #Gates parameters for bias:
  #ib = tf.Variable(tf.zeros([1, num_nodes]))
  #cb = tf.Variable(tf.zeros([1, num_nodes]))
  #fb = tf.Variable(tf.zeros([1, num_nodes]))
  #ob = tf.Variable(tf.zeros([1, num_nodes]))
  gb = tf.Variable(tf.zeros([num_gates, 1, num_nodes]))

  # Variables saving state across unrollings.
  saved_omem = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, last_state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the previous state and the gates."""
    
    I = tf.stack([i,i,i,i])
    O = tf.stack([o,o,o,o])
    gates = tf.matmul(I, gx) + tf.matmul(O, gm) + gb
    
    #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    
    update = gates[0,:,:]
    input_gate = tf.sigmoid(gates[1,:,:])
    forget_gate = tf.sigmoid(gates[2,:,:])
    output_gate = tf.sigmoid(gates[3,:,:])
    
    next_state = forget_gate * last_state + input_gate * tf.tanh(update)
    outputmem = output_gate * tf.tanh(next_state)
    
    return outputmem, next_state

  # Input data, get input and labels.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  omemories = list()
  omem = saved_omem
  state = saved_state
  for i in train_inputs:
    omem, state = lstm_cell(i, omem, state)
    omemories.append(omem)

  # State saving across unrollings.
  with tf.control_dependencies([saved_omem.assign(omem),saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(omemories, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat_v2(train_labels, 0)))
                #Evalúa la salida de cada ráfaga con un logistic classifier normal

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, learning_decay_steps, learning_decay_rate, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, clip_limit) #Limitar los pesos para que no se disparen y quitar el exploding
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch size = 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_omem = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(saved_sample_omem.assign(tf.zeros([1, num_nodes])),saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_omem, sample_state = lstm_cell(sample_input, saved_sample_omem, saved_sample_state)
  with tf.control_dependencies([saved_sample_omem.assign(sample_omem),saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_omem, w, b))
    
    #Para verificar, toma nuestro texto de validación y ve si se predice bien aplicando X'*W+b = Y, 
    #donde X' es la salida de la MEMORIA después de meterle X de entrada y el estado anterior guardado S (HISTORIA)
    #El texto de validación se alimenta letra a letra, no por ráfagas como el entrenamiento

In [10]:
##################
# EJECUCION LTSM1
##################
t1 = time.time()

num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graphLSTM1) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l

    #Informe
    if step % summary_frequency == 0:
      t2 = time.time()
      # The mean loss is an estimate of the loss over the last few batches.
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # La preplejidad depende de cuanto acierta con las etiquetas
      labels = np.concatenate(list(batches)[1:])
      mbpx = float(np.exp(logprob(predictions, labels)))
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      vspx = float(np.exp(valid_logprob / valid_size))
      #Report
      print('step\t%d\t%ds:\tAvgLoss %f\tLRate %f\tMBperplex %.2f\tVSperplex %.2f' % (step, t2-t1, mean_loss, lr, mbpx, vspx))
      mean_loss = 0
    
      #Informe más completo con muestras
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
  print("End.")

Initialized
step	0	3s:	AvgLoss 3.295826	LRate 10.000000	MBperplex 27.00	VSperplex 20.25
gpgcujlwpfsbnh roa iuoshyvj om ih keylzwmzcpst  rbp  a r vhtjzs tuc kd lheetczxv
vdg fllnifpmaaii sm taihrrmxeh bgl mea nei to ognr xcnetp j nilnimntoypmsdqnwd  
jq e  fi er  neqcgsqpefxahyeqtipi xnf dem nqkoynyztx tqudyrfjcsr raht t ftvuzifl
xkadu rejyjlxovfuftsieel ue phace uoriee ze joo uo ieltmcnhandko immeenhqjlpm  z
rsimmseuo iocksgii fatxdej uixr ep z  cqcldhh kxwcaesvwt dbtd c dct uledrjprotkn
step	100	7s:	AvgLoss 2.594235	LRate 10.000000	MBperplex 10.97	VSperplex 10.33
step	200	11s:	AvgLoss 2.246921	LRate 10.000000	MBperplex 8.57	VSperplex 8.43
step	300	16s:	AvgLoss 2.096534	LRate 10.000000	MBperplex 7.48	VSperplex 8.25
step	400	20s:	AvgLoss 1.998788	LRate 10.000000	MBperplex 7.55	VSperplex 7.72
step	500	24s:	AvgLoss 1.931895	LRate 10.000000	MBperplex 6.49	VSperplex 7.10
step	600	28s:	AvgLoss 1.900794	LRate 10.000000	MBperplex 6.18	VSperplex 6.88
step	700	33s:	AvgLoss 1.850499	LRate 10.0000